# Power Curve App

It’ll be served the app with:
```
panel serve app.ipynb --autoreload
```
Now, open the app in your browser at **http://localhost:5006/app**.

### References

- https://coderzcolumn.com/tutorials/data-science/basic-dashboard-using-panel


In [1]:
import os
import numpy as np
import pandas as pd
import panel as pn
import holoviews as hv
from datetime import datetime, date
import hvplot.pandas

## arguments

In [2]:
PRIMARY_COLOR = "#0072B5"
SECONDARY_COLOR = "#B54300"
FOLDER_DATA= (
    "/Users/juan/Workspace/data/datasets/dashboards"
)

## initialize

In [3]:
pn.extension(design="material", sizing_mode="stretch_width")
hv.extension("bokeh")

## data loading

In [4]:
# collect folder data options
data_options = [folder for folder in os.listdir(FOLDER_DATA) if folder != '.DS_Store']
# widget data option selection
folder_option_widget = pn.widgets.Select(name="Folder Option", value=data_options[0], options=list(data_options))

In [5]:
#@pn.cache
def load_data(folder_option):
    folder = os.path.join(FOLDER_DATA, folder_option)
    files = [file for file in os.listdir(folder) if ".csv" in file]
    df = pd.DataFrame()
    for file in files:
        path = os.path.join(folder, file)
        df = pd.concat([df, pd.read_csv(path)], axis = 0)
    return df.dropna()

def date_to_datetime(dt):
    return datetime(dt.year, dt.month, dt.day, 0, 0, 0)

def transform_data(data):
    data.set_index("Time", inplace = True)
    data.WindSpeed = data.WindSpeed.astype(float) 
    data.Power = data.Power.astype(float) 
    data.index = pd.to_datetime(data.index)
    return data


In [6]:
folder_option_widget

BokehModel(combine_events=True, render_bundle={'docs_json': {'44c377b1-6d70-478b-88cb-35fe16a12c89': {'version…

In [7]:
# load data
data = load_data(folder_option_widget.value)
# data transform
data = transform_data(data)
# display
data.shape

(7265, 2)

## plots

In [8]:
variable = 'WindSpeed'
plot_timeseries_wind = data.hvplot(y = variable, height=300, legend=False, color=PRIMARY_COLOR)
plot_timeseries_wind

:Curve   [Time]   (WindSpeed)

In [9]:
variable = 'Power'
plot_timeseries_power = data.hvplot(y = variable,height=300, legend=False, color=PRIMARY_COLOR)
plot_timeseries_power

:Curve   [Time]   (Power)

In [10]:
plot_scatter_pc = data.hvplot.scatter(x='WindSpeed', y='Power', color=PRIMARY_COLOR, padding=0.1, legend=False, size = 10)
plot_scatter_pc

:Scatter   [WindSpeed]   (Power)

### linking the plots

In [11]:
ls = hv.link_selections.instance()
linked_plots = ls(plot_scatter_pc + plot_timeseries_wind + plot_timeseries_power).cols(1)
linked_plots

BokehModel(combine_events=True, render_bundle={'docs_json': {'1c9a69db-7ff5-4552-9eec-d4ae842af0e9': {'version…

## building a dashboard

In [12]:
# selected data counter
@pn.depends(ls.selection_param(data))
def count(selected):
    return pn.pane.Markdown(f"## Selected {len(selected)}/{len(data)} points.", align = 'center')
pn.panel(count)

BokehModel(combine_events=True, render_bundle={'docs_json': {'6083e2d3-924c-4d14-8514-a7712cec6651': {'version…

In [13]:
# welcome message
welcome = "## Welcome to this dashboard"
# build header
header = pn.Row(welcome, count)
header

BokehModel(combine_events=True, render_bundle={'docs_json': {'4d6e6deb-78ce-4967-9170-de242d00456f': {'version…

## deploying the dashboard

In [14]:
material = pn.template.MaterialTemplate(
    site="Panel",
    title="Prueba App",
    #sidebar=[folder_option_widget, start_date_picker, end_date_picker],
    #main=[date_range_slider, bound_plot_1, bound_plot_2, bound_plot_3],
)

folder_option_widget = pn.widgets.Select(name="Folder Option", value=data_options[0], options=list(data_options))

@pn.depends(folder_option = folder_option_widget)
def plots(folder_option):
    data = load_data(folder_option)
    data = transform_data(data)
    plot_timeseries_wind = data.hvplot(y = "WindSpeed", height=200, legend=False, color=PRIMARY_COLOR, responsive = True)
    plot_timeseries_power = data.hvplot(y = "Power", height=200, legend=False, color=PRIMARY_COLOR, responsive = True)
    plot_scatter_pc = data.hvplot.points(x='WindSpeed', y='Power', color=PRIMARY_COLOR, padding=0.1, legend=False, size = 10, height=400, responsive = True)
    ls = hv.link_selections.instance()
    plots =  ls(plot_scatter_pc + plot_timeseries_wind + plot_timeseries_power).cols(1)  
    #plots.opts(tools=["hover"], active_tools=["box_select"])
    #https://github.com/holoviz/holoviews/issues/5488
    return plots
#@pn.depends(ls.selection_param(data))
#def count(selected):
#    return pn.pane.Markdown(f"## Selected {len(selected)}/{len(data)} points.", align = 'center')
header = pn.Row(welcome, count)
material.header.append(header)
material.sidebar.append(folder_option_widget)
material.main.append(pn.Row(plots))

material.show()
#material.servable();

Launching server at http://localhost:49607
